# MNIST Partial Questing - with Curves! For longer!

Increasing training time for this model.

## Imports

In [ ]:
import random

import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from helpers import load_images, load_labels, visualize_image, get_edges, generate_intermediate_edge_labels, horizontal_kernel, vertical_kernel, \
    load_intermediate_labels, generate_intermediate_curve_labels, save_intermediate_labels

## Globals

In [ ]:
MODEL_NAME = "./curve_partial_long.pth"

## Set Device to GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"We will be using device: {device}")

## Load Data

In [ ]:
# train data
train_images = load_images("./data/train-images-idx3-ubyte/train-images-idx3-ubyte")
train_labels = load_labels("./data/train-labels-idx1-ubyte/train-labels-idx1-ubyte")

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels,
    test_size=1/6,  # 10k validation
    stratify=train_labels,
    random_state=42  # for reproducibility
)

# test data
test_images = load_images("./data/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte")
test_labels = load_labels("./data/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte")

In [ ]:
print("Train images shape:", train_images.shape)
print("Val images shape:", val_images.shape)
print("Test images shape:", test_images.shape)

## Visualize an Image

In [ ]:
sample_image = train_images[0]
sample_label = train_labels[0]
visualize_image(sample_image, sample_label)

## Convolutions to get Horizontal Edges

### Example Image

In [ ]:
horizontal_edges = get_edges(horizontal_kernel, sample_image)

In [ ]:
visualize_image(horizontal_edges, f"{sample_label} horizontal edges")

### Extending the Dataset

In [ ]:
train_horizontal_image_labels = load_intermediate_labels("train_horizontal_image_labels.pkl")

In [ ]:
val_horizontal_image_labels = load_intermediate_labels("val_horizontal_image_labels.pkl")

In [ ]:
test_horizontal_image_labels = load_intermediate_labels("test_horizontal_image_labels.pkl")

In [ ]:
visualize_image(train_horizontal_image_labels[0].reshape(28, 28), "")

## Convolutions to get Vertical Edges

### Example Image

In [ ]:
vertical_edges = get_edges(vertical_kernel, sample_image)

In [ ]:
visualize_image(vertical_edges, f"{sample_label} vertical edges")

### Extending the Dataset

In [ ]:
train_vertical_image_labels = load_intermediate_labels("train_vertical_image_labels.pkl")

In [ ]:
val_vertical_image_labels = load_intermediate_labels("val_vertical_image_labels.pkl")

In [ ]:
test_vertical_image_labels = load_intermediate_labels("test_vertical_image_labels.pkl")

In [ ]:
visualize_image(train_vertical_image_labels[0].reshape(28, 28), "")

## Convolutions to get Curves

Some extra steps here to generate better curves... this might be a point to re-visit in testing, but it largely comes from needing to rotate our curve kernel to capture curves.

The point to revisit is perhaps we want each unique rotation to be represented by an intermediate layer. Similar to how edges have been split into vertical and horizontal layers, perhaps we want layers to be dedicated to only one curve, instead of the culmination of curves.

### Example Image

In [ ]:
curves = generate_intermediate_curve_labels([sample_image])

In [ ]:
visualize_image(curves[0].reshape(28, 28), f"{sample_label} curves")

### Extending the dataset

In [ ]:
train_curve_labels = load_intermediate_labels("train_curve_labels.pkl")

In [ ]:
val_curve_labels = load_intermediate_labels("val_curve_labels.pkl")

In [ ]:
test_curve_labels = load_intermediate_labels("test_curve_labels.pkl")

In [ ]:
visualize_image(train_curve_labels[0].reshape(28, 28), "")

# Base Test

## Architecture

Updating the architecture to take into account our new curve layer.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        layer_size_by_pixels = 28*28
        
        self.flatten = nn.Flatten()
        
        # define layers separately to have access to each
        self.horizontal_layer = nn.Linear(layer_size_by_pixels, layer_size_by_pixels)
        self.vertical_layer = nn.Linear(layer_size_by_pixels, layer_size_by_pixels)
        self.curve_layer = nn.Linear(layer_size_by_pixels, layer_size_by_pixels)
        self.classification_layer = nn.Linear(layer_size_by_pixels, 10)
        self.activation_function = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)

        # horizontal processing
        horizontal_out = self.horizontal_layer(x)
        horizontal_act = self.activation_function(horizontal_out)

        # vertical processing
        vertical_out = self.vertical_layer(horizontal_act)
        vertical_act = self.activation_function(vertical_out)

        # curve processing
        curve_out = self.curve_layer(vertical_act)
        curve_act = self.activation_function(curve_out)

        # classification
        classification_out = self.classification_layer(curve_act)
        
        return classification_out, curve_act, vertical_act, horizontal_act

In [ ]:
# for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
model = NeuralNetwork().to(device)

# loss functions
classification_loss_fn = nn.CrossEntropyLoss()
horizontal_loss_fn = nn.MSELoss()
vertical_loss_fn = nn.MSELoss()
curve_loss_fn = nn.MSELoss()

# optimizers
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
print(f"Model weights (to compare below): {model.horizontal_layer.weight[0][:5].detach().cpu().numpy()}")

### Verify Reproducibility

In [ ]:
# reset the seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    
model_compare_one = NeuralNetwork().to(device)
first_set_of_weights = model_compare_one.horizontal_layer.weight[0][:5].detach().cpu().numpy()
print("First set of weights:", first_set_of_weights)

# reset the seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

model_compare_two = NeuralNetwork().to(device)
second_set_of_weights = model_compare_two.horizontal_layer.weight[0][:5].detach().cpu().numpy()
print("Second set of weights:", second_set_of_weights)

print(f"Are the two sets equal: {first_set_of_weights == second_set_of_weights}")

## Dataloader

In [ ]:
class EdgeDataset(Dataset):
    def __init__(self, images, labels, horizontal_edges, vertical_edges, curves):
        self.images = images
        self.labels = labels
        self.horizontal_edges = horizontal_edges
        self.vertical_edges = vertical_edges
        self.curves = curves

    def __len__(self):
        assert len(self.images) == len(self.labels)
        return len(self.images)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.images[idx].copy()).float(),
            torch.tensor(self.labels[idx].copy(), dtype=torch.long),
            torch.from_numpy(self.horizontal_edges[idx].copy()).float(),
            torch.from_numpy(self.vertical_edges[idx].copy()).float(),
            torch.from_numpy(self.curves[idx].copy()).float(),
        )

In [ ]:
# for reproducibility on training
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

generator = torch.Generator().manual_seed(seed)

In [ ]:
# training data
train_dataset = EdgeDataset(train_images, train_labels, train_horizontal_image_labels, train_vertical_image_labels, train_curve_labels)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, worker_init_fn=seed_worker, generator=generator)

# validation data
val_dataset = EdgeDataset(val_images, val_labels, val_horizontal_image_labels, val_vertical_image_labels, val_curve_labels)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)  # larger batch size for faster validation

# test data
test_dataset = EdgeDataset(test_images, test_labels, test_horizontal_image_labels, test_vertical_image_labels, test_curve_labels)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

## Training Loop

In [ ]:
num_epochs = 150
best_val_loss = float('inf')
for epoch in range(num_epochs):
    # --- Training Phase ---
    model.train()  # set the model to training mode - this is currently a no-op
    train_loss = 0.0

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} [Train]", leave=False)
    for batch in train_bar:
        # deconstruct batch items
        images, labels, horizontal_labels, vertical_labels, curve_labels = batch
        images, labels, horizontal_labels, vertical_labels, curve_labels = images.to(device), \
            labels.to(device), \
            horizontal_labels.to(device), \
            vertical_labels.to(device), \
            curve_labels.to(device)
        
        # forward pass
        classification_out, curve_act, vertical_act, horizontal_act = model(images)
        
        # --- Loss and Backprop ---

        # clear gradients
        optimizer.zero_grad()

        # curve_loss 
        curve_loss = curve_loss_fn(curve_act, curve_labels)
        
        # vertical loss
        vertical_loss = vertical_loss_fn(vertical_act, vertical_labels)

        # horizontal loss
        horizontal_loss = horizontal_loss_fn(horizontal_act, horizontal_labels)

        # classification loss
        classification_loss = classification_loss_fn(classification_out, labels)

        # total loss
        total_loss = curve_loss + vertical_loss + horizontal_loss + classification_loss
        total_loss.backward()
        
        optimizer.step()

        # update progress
        train_loss += total_loss.item()
        train_bar.set_postfix(loss=classification_loss.item())

    # --- Validation Phase ---
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} [Val]", leave=False)
    with torch.no_grad():
        for batch in val_bar:
            # deconstruct
            images, labels, _, _, _ = batch
            images, labels = images.to(device), labels.to(device)

            # forward pass
            classification_out, _, _, _ = model(images)

            # compute loss
            loss = classification_loss_fn(classification_out, labels)

            # calculate metrics
            val_loss += loss.item()
            _, predicted = torch.max(classification_out, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    # epoch stats
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")

    # save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), MODEL_NAME)

So our training doesn't seem to really matter since loss is best in the first step. This appears to by a byproduct of our loss defintion in the training loop vs. validation loop. Ideally, we'd want the classification loss to be minimized. Perhaps we should experiment to see if minimizing training loss can help? I suspect it won't, since the side quests might counter the main quests, or we might get stuck in oscillation.

## Eval

In [ ]:
model.load_state_dict(torch.load(MODEL_NAME))
model.eval()  # again currently a no-op

test_correct = 0
test_total = 0

with torch.no_grad():
    test_bar = tqdm(test_loader, desc=f"Evaluation")
    for batch in test_bar:
        images, labels, _, _, _ = batch
        images, labels = images.to(device), labels.to(device)

        # forward pass
        classification_out, _, _, _ = model(images)

        # stats
        _, predicted = torch.max(classification_out, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_accuracy = 100 * test_correct / test_total
print(f"Final Test Accuracy: {test_accuracy:.2f}%")

# Exploring Model Results

## Visualizing Weights

In [ ]:
horizontal_layer_weights = np.abs(model.horizontal_layer.weight[0].reshape(28, 28).detach().cpu().numpy())

In [ ]:
visualize_image(horizontal_layer_weights, "Horizontal Layer Weights")

In [ ]:
vertical_layer_weights = np.abs(model.vertical_layer.weight[0].reshape(28, 28).detach().cpu().numpy())

In [ ]:
visualize_image(vertical_layer_weights, "Vertical Layer Weights")

In [ ]:
curve_layer_weights = np.abs(model.curve_layer.weight[0].reshape(28, 28).detach().cpu().numpy())

In [ ]:
visualize_image(curve_layer_weights, "Curve Layer Weights")

In [ ]:
classification_layer_weights = np.abs(model.classification_layer.weight[0].reshape(28, 28).detach().cpu().numpy())

In [ ]:
visualize_image(classification_layer_weights, "Classification Layer Weights")